In [1]:
import tensorflow as tf
tf.VERSION

'1.14.0'

In [2]:
import pandas as pd
import os
import tensorflow as tf
# import cv2 as cv
import numpy as np
import glob
# # from random import shuffle
#from google.colab import drive
import matplotlib.pyplot as plt
import csv
import re
import json
import math
from sklearn.utils import shuffle
import tensorflow_hub as hub
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding,Input, Flatten, Activation
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Bidirectional,LSTM,GlobalMaxPool1D,Dense,Concatenate
from keras.utils.vis_utils import plot_model
from keras import losses
from keras.layers.merge import Multiply,Dot
import keras.backend as K


Using TensorFlow backend.


In [3]:
dataset = pd.read_csv('video_LLDs_FAUs/test_01_OpenFace2.1.0_Pose_gaze_AUs.csv')

dataset.columns

Index(['frame', 'timestamp', 'confidence', 'success', 'pose_Tx', 'pose_Ty',
       'pose_Tz', 'pose_Rx', 'pose_Ry', 'pose_Rz', 'gaze_0_x', 'gaze_0_y',
       'gaze_0_z', 'gaze_1_x', 'gaze_1_y', 'gaze_1_z', 'gaze_angle_x',
       'gaze_angle_y', 'AU01_r', 'AU02_r', 'AU04_r', 'AU05_r', 'AU06_r',
       'AU07_r', 'AU09_r', 'AU10_r', 'AU12_r', 'AU14_r', 'AU15_r', 'AU17_r',
       'AU20_r', 'AU23_r', 'AU25_r', 'AU26_r', 'AU45_r', 'AU01_c', 'AU02_c',
       'AU04_c', 'AU05_c', 'AU06_c', 'AU07_c', 'AU09_c', 'AU10_c', 'AU12_c',
       'AU14_c', 'AU15_c', 'AU17_c', 'AU20_c', 'AU23_c', 'AU25_c', 'AU26_c',
       'AU28_c', 'AU45_c'],
      dtype='object')

In [4]:
metadata = pd.read_csv('metadata.csv')
sample = pd.read_csv('video_LLDs_FAUs/training_001_OpenFace2.1.0_Pose_gaze_AUs.csv')
sample_bow = pd.read_csv('video_BoVW/development_01_BoVW_openFace_2.1.0_Pose_Gaze_AUs.csv', header = None, delimiter = ';')


In [5]:
bow_cols = sample_bow.columns[2:102]

In [6]:
path = 'video_BoVW/'
bow_train = []
c = 0
for i in metadata['Participant_ID']:

  for file in os.listdir(path):
    
    if i.startswith('train') and file.startswith(i):
      c+=1
#       print(i)
#       print(c)
      
#       print(file[0:4]=='trai'):
        
      data = pd.read_csv('video_BoVW/'+file, header = None, delimiter=';')
      data = data[bow_cols]
      bow_train.append(data)

pose_cols = sample.columns[4:10]
gaze_cols = sample.columns[10:18]
au_cols = sample.columns[18:]

In [7]:
path = 'video_LLDs_FAUs/'
pose_train = []
gaze_train = []
au_train = []
c = 0
for i in metadata['Participant_ID']:

  for file in os.listdir(path):
    
    if i.startswith('train') and file.startswith(i):
      c+=1
#       print(i)
#       print(c)
      
#       print(file[0:4]=='trai'):
        
      feat = pd.read_csv('video_LLDs_FAUs/'+file)
      feat = feat.iloc[::3, :]
      pose_train.append(feat[pose_cols])
      gaze_train.append(feat[gaze_cols])
      au_train.append(feat[au_cols])

In [8]:
path = 'video_BoVW/'
bow_dev = []
c = 0
for i in metadata['Participant_ID']:

  for file in os.listdir(path):
    
    if i.startswith('development') and file.startswith(i):
      c+=1
      #print(i)
     # print(c)
      
#       print(file[0:4]=='trai'):
        
      data = pd.read_csv('video_BoVW/'+file, header = None, delimiter=';')
      data = data[bow_cols]
      bow_dev.append(data)

In [9]:
path = 'video_LLDs_FAUs/'
pose_dev = []
gaze_dev = []
au_dev = []
c = 0
for i in metadata['Participant_ID']:

  for file in os.listdir(path):
    
    if i.startswith('dev') and file.startswith(i):
      c+=1
      print(i,c)
      
#       print(file[0:4]=='trai'):
        
      feat = pd.read_csv('video_LLDs_FAUs/'+file)
      feat = feat.iloc[::3, :]
      pose_dev.append(feat[pose_cols])
      gaze_dev.append(feat[gaze_cols])
      au_dev.append(feat[au_cols])

development_01 1
development_02 2
development_03 3
development_04 4
development_05 5
development_06 6
development_07 7
development_08 8
development_09 9
development_10 10
development_11 11
development_12 12
development_13 13
development_14 14
development_15 15
development_16 16
development_17 17
development_18 18
development_19 19
development_20 20
development_21 21
development_22 22
development_23 23
development_24 24
development_25 25
development_26 26
development_27 27
development_28 28
development_29 29
development_30 30
development_31 31
development_32 32
development_33 33
development_34 34
development_35 35
development_36 36
development_37 37
development_38 38
development_39 39
development_40 40
development_41 41
development_42 42
development_43 43
development_44 44
development_45 45
development_46 46
development_47 47
development_48 48
development_49 49
development_50 50
development_51 51
development_52 52
development_53 53
development_54 54
development_55 55
development_56 56


In [10]:
def embed_useT(module):
    with tf.Graph().as_default():
        sentences = tf.placeholder(tf.string)
        embed = hub.Module(module)
        embeddings = embed(sentences)
        session = tf.train.MonitoredSession()
    return lambda x: session.run(embeddings, {sentences: x})

In [11]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm","i am",text)
    text = re.sub(r"i've","i have",text)
    text = re.sub(r"he's","he is",text)
    text = re.sub(r"she's","she is",text)
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"that ' s","that is",text)
    text = re.sub(r"it's","it is",text)
    text = re.sub(r"that's","that is",text)
    text = re.sub(r"where's","where is",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"\'ll"," will",text) 
    text = re.sub(r"\'ve"," have",text)
    text = re.sub(r"\'re"," are",text)
    text = re.sub(r"\'d"," would",text)  
    text = re.sub(r"won't", "will not",text)
    text = re.sub(r"don't", "do not",text)
    text = re.sub(r"can't","can not", text)
    text = re.sub(r"hadn't","had not", text)
    text = re.sub(r"didn't","did not", text)
    text = re.sub(r"wouldn't","would not", text)
    text = re.sub(r"weren't","were not", text)
    text = re.sub(r"shouldn't","should not", text)
    text = re.sub(r"doesn't","does not", text)
    text = re.sub(r"couldn't","could not", text)
    text = re.sub(r"isn't","is not", text)
    text = re.sub(r"hasn't","has not", text)
    text = re.sub(r"wasn't","was not", text)
    text = re.sub(r"haven't","have not", text)
    text = re.sub(r"didn't","did not", text)
    text = re.sub(r"wouldnt'","would not", text)
#     text = re.sub(r"shouldnt'","should not", text)
#     text = re.sub(r"doesnt'","does not", text)
#     text = re.sub(r"couldnt'","could not", text)
#     text = re.sub(r"isnt'","is not", text)
#     text = re.sub(r"wasn't","was not", text)
#     text = re.sub(r"havent'","have not", text)
    text = re.sub(r"aren't","are not", text)
#     text = re.sub(r"werent","were not", text)
    text = re.sub(r" em "," them ", text)
    text = re.sub(r" there's "," there is ", text)
    text = re.sub(r"let's","let us", text)
    text = re.sub(r" who's "," who is ", text)
    text = re.sub(r"\'s","", text)
    text = re.sub(r"'","", text)
# #     text = re.sub(r"arent","are not", text)
#     text = re.sub(r"  "," ", text)
#     text = re.sub(r"[-?!@#$%^&*(~,;)+=_`<>{}.\"']","",text)
#     pattern = r'[^a-zA-z0-9\s]'
#     text = re.sub(pattern, '', text)
    return text

In [12]:
def sent(data,cnt):
#   Participant_transcript=[rows[0].split("\t") for rows in data if len(rows)>0 if rows[0].split("\t")[2] == "Participant" ]
    if cnt==0:
      Participant_transcript=[rows[0].split("\t") for rows in data if len(rows)>0 if rows[0].split("\t")[2] == "Participant" ]
#   print(Participant_transcript)
      words=[clean_text(x[3].strip()).strip() for x in Participant_transcript]
    else:
      words=[clean_text(x[2].strip()).strip() for x in data]
      words.pop(0)
#     print(words)
    for ind,val in enumerate(words):
        a=val.split(' ')
        for ii,tp in enumerate(a):
            if(len(tp)):
                if(tp[0]=='<'and tp[-1]=='>'):
                    a[ii]=tp[1:-1]
                elif(tp[0]=='<' ):
                    a[ii]=tp[1:]
        words[ind]=' '.join(a).strip()        
    m.append(len(words))
#     print(words)
    return words

In [13]:
tf.VERSION
sentences=[]
target_asr=[]
m=[]
embed_fn = embed_useT('USE/')
embedding_matrix=[]

W0713 00:24:12.409619 140520442545920 deprecation.py:323] From /home/siddharth/my_env/lib/python3.6/site-packages/tensorflow/python/ops/array_ops.py:1354: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [14]:
emb_dim_asr=400
pth=["ASR_transcripts/*.csv"]
for cnt,p in enumerate(pth):
  cnt+=1
  if cnt>0:
    for filename in glob.glob(p):#CHANGE PATH
        print(filename)
        if 'test' not in filename:
          if(cnt==0):
            target_asr.append(filename.split("/")[-1].split("_")[0])
          else:
            target_asr.append(filename.split("/")[-1].split(".")[0][:-11])
          with open(filename,"rt") as f:
            data=csv.reader(f)
            msg_txt=sent(data,cnt)
      #       print(msg_txt)  
            emb=embed_fn(msg_txt)
            a = np.zeros((emb_dim_asr-emb.shape[0], 512))
            x=np.vstack((a,emb))
            embedding_matrix.append(x)
            print(emb.shape)

ASR_transcripts/development_43_Transcript.csv
(106, 512)
ASR_transcripts/training_040_Transcript.csv
(64, 512)
ASR_transcripts/test_40_Transcript.csv
ASR_transcripts/training_039_Transcript.csv
(44, 512)
ASR_transcripts/test_21_Transcript.csv
ASR_transcripts/test_25_Transcript.csv
ASR_transcripts/training_062_Transcript.csv
(57, 512)
ASR_transcripts/test_34_Transcript.csv
ASR_transcripts/training_089_Transcript.csv
(80, 512)
ASR_transcripts/training_049_Transcript.csv
(64, 512)
ASR_transcripts/development_25_Transcript.csv
(91, 512)
ASR_transcripts/training_104_Transcript.csv
(47, 512)
ASR_transcripts/training_135_Transcript.csv
(91, 512)
ASR_transcripts/development_34_Transcript.csv
(70, 512)
ASR_transcripts/test_18_Transcript.csv
ASR_transcripts/training_128_Transcript.csv
(64, 512)
ASR_transcripts/training_080_Transcript.csv
(165, 512)
ASR_transcripts/training_098_Transcript.csv
(73, 512)
ASR_transcripts/development_56_Transcript.csv
(111, 512)
ASR_transcripts/training_063_Transcrip

(74, 512)
ASR_transcripts/training_053_Transcript.csv
(131, 512)
ASR_transcripts/development_15_Transcript.csv
(117, 512)
ASR_transcripts/training_078_Transcript.csv
(65, 512)
ASR_transcripts/test_45_Transcript.csv
ASR_transcripts/test_11_Transcript.csv
ASR_transcripts/development_19_Transcript.csv
(107, 512)
ASR_transcripts/training_116_Transcript.csv
(71, 512)
ASR_transcripts/training_079_Transcript.csv
(164, 512)
ASR_transcripts/development_07_Transcript.csv
(69, 512)
ASR_transcripts/development_41_Transcript.csv
(60, 512)
ASR_transcripts/training_004_Transcript.csv
(203, 512)
ASR_transcripts/development_29_Transcript.csv
(82, 512)
ASR_transcripts/development_18_Transcript.csv
(141, 512)
ASR_transcripts/training_120_Transcript.csv
(127, 512)
ASR_transcripts/test_09_Transcript.csv
ASR_transcripts/training_086_Transcript.csv
(92, 512)
ASR_transcripts/training_074_Transcript.csv
(75, 512)
ASR_transcripts/training_093_Transcript.csv
(90, 512)
ASR_transcripts/training_092_Transcript.csv


In [15]:
embedding_matrix=np.asarray(embedding_matrix)

In [16]:
print(embedding_matrix.shape)

(220, 400, 512)


In [17]:
train_data_asr=[]
pth=["metadata.csv"]
for p in pth:
    with open(p,"rt") as f:
        l_data=csv.reader(f)
        for r in l_data:
          if len(r):
            if r[0] in target_asr and ('training' in r[0]):
              train_data_asr.append(embedding_matrix[target_asr.index(r[0])])


In [18]:
train_data_asr=np.asarray(train_data_asr,dtype=np.float32)

In [19]:
val_data_asr=[]
pth=["metadata.csv"]
for p in pth:
    with open(p,"rt") as f:
        l_data=csv.reader(f)
        for r in l_data:
          if len(r):
            if r[0] in target_asr and ('development' in r[0]):
              val_data_asr.append(embedding_matrix[target_asr.index(r[0])])


In [20]:
val_data_asr=np.asarray(val_data_asr,dtype=np.float32)

In [21]:
print(train_data_asr.shape)
print(val_data_asr.shape)

(163, 400, 512)
(56, 400, 512)


In [22]:
target = []

for i,v in metadata.iterrows():
  if v['Participant_ID'].startswith('train'):
    target.append(v['PHQ_Score'])

dev_target = []

for i,v in metadata.iterrows():
  if v['Participant_ID'].startswith('dev'):
    dev_target.append(v['PHQ_Score'])

for i in pose_train:
#   print(i)
  i['pose_Tx'] = i['pose_Tx']/100
  i['pose_Ty'] = i['pose_Ty']/100
  i['pose_Tz'] = i['pose_Tz']/100

# c = 0
# for i in pose_test:
#   print(c, end = ' ')
#   c+=1
#   print(i)
  i['pose_Tx'] = i['pose_Tx']/100
  i['pose_Ty'] = i['pose_Ty']/100
  i['pose_Tz'] = i['pose_Tz']/100

c = 0
for i in pose_dev:
  print(c,end = ' ')
  c+=1
#   print(i)
  i['pose_Tx'] = i['pose_Tx']/100
  i['pose_Ty'] = i['pose_Ty']/100
  i['pose_Tz'] = i['pose_Tz']/100


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 

/home/siddharth/my_env/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/siddharth/my_env/lib/python3.6/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/siddharth/my_env/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

In [23]:
MAX_SEQ_LEN = 15000

train_seq_pose = pad_sequences(pose_train, maxlen=MAX_SEQ_LEN,dtype = 'float32')
# test_seq_pose = pad_sequences(pose_test, maxlen=MAX_SEQ_LEN,dtype = 'float32')
dev_seq_pose = pad_sequences(pose_dev, maxlen=MAX_SEQ_LEN,dtype = 'float32')

train_seq_gaze = pad_sequences(gaze_train, maxlen=MAX_SEQ_LEN,dtype = 'float32')
# test_seq_gaze = pad_sequences(gaze_test, maxlen=MAX_SEQ_LEN,dtype = 'float32')
dev_seq_gaze =  pad_sequences(gaze_dev, maxlen=MAX_SEQ_LEN,dtype = 'float32')

train_seq_au = pad_sequences(au_train, maxlen=MAX_SEQ_LEN,dtype = 'float32')
# test_seq_au = pad_sequences(au_test, maxlen=MAX_SEQ_LEN,dtype = 'float32')
dev_seq_au = pad_sequences(au_dev, maxlen=MAX_SEQ_LEN,dtype = 'float32')

train_seq_bow = pad_sequences(bow_train, maxlen=MAX_SEQ_LEN,dtype = 'float32')
# test_seq_bow = pad_sequences(bow_test, maxlen=MAX_SEQ_LEN,dtype = 'float32')
dev_seq_bow = pad_sequences(bow_dev, maxlen=MAX_SEQ_LEN,dtype = 'float32')


In [24]:
target = np.reshape(target,[-1,1])
dev_target = np.reshape(dev_target,[-1,1])

In [25]:
# def feedf_network(xx,dp):
#   with tf.variable_scope('FUSION'):
#     logits=tf.layers.batch_normalization(tf.nn.relu(tf.layers.dense(xx,500)))
#     logits=tf.nn.dropout(logits,keep_prob=dp)
#     logits=tf.layers.batch_normalization(tf.nn.relu(tf.layers.dense(logits,100)))
#     logits=tf.nn.dropout(logits,keep_prob=dp)
#     logits=tf.layers.batch_normalization(tf.nn.relu(tf.layers.dense(logits,60)))
#     logits= tf.layers.dense(logits,1)
    
#     return logits

In [26]:
# tf.reset_default_graph()
# print("network started")
# true_targets = tf.placeholder(shape=(None,1),dtype = tf.float32)

# input_bow = tf.placeholder(shape=(None,MAX_SEQ_LEN,100),dtype = tf.float32)
# context_layer_bow = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences = False))(input_bow)
# print('bow',context_layer_bow.shape)
# input_pose = tf.placeholder(shape=(None,MAX_SEQ_LEN,6),dtype = tf.float32)
# context_layer_pose = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences = False))(input_pose)
# print('pose',context_layer_pose.shape)

# input_gaze = tf.placeholder(shape=(None,MAX_SEQ_LEN,8),dtype = tf.float32, name='GAZE')
# context_layer_gaze = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences = False))(input_gaze)
# print('gaze',context_layer_gaze.shape)

# input_au = tf.placeholder(shape=(None,MAX_SEQ_LEN,35),dtype = tf.float32)
# context_layer_au = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences = False))(input_au)
# print('au',context_layer_au.shape)
# ############################################################################# asr

# X_asr=tf.placeholder(tf.float32,[None,emb_dim_asr,512])
# mx_asr=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences=True))(X_asr)
# print('mx',mx_asr.shape)
# context_X_asr=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences=True))(mx_asr)
# print('context_asr',context_X_asr.shape)
# context_X_asr=tf.reduce_sum(context_X_asr,1)
# print('asr',context_X_asr.shape)
#  ##############################################################################################
# fusion_inp=tf.concat([context_X_asr,context_layer_bow,context_layer_pose,context_layer_gaze,
#                       context_layer_au], 1)
# print('fusion',fusion_inp.shape)


# fused_logits=feedf_network(fusion_inp,1.0)

# loss=tf.reduce_mean(tf.losses.mean_squared_error(labels=true_targets,predictions=fused_logits))
# optimizer = tf.train.AdamOptimizer(learning_rate = 0.01).minimize(loss)
# print('graph ready')

In [27]:
#NETWORK DEFINITION
tf.reset_default_graph()
print("network started")
true_targets = tf.placeholder(shape=(None,1),dtype = tf.float32)

input_bow = tf.placeholder(shape=(None,MAX_SEQ_LEN,100),dtype = tf.float32)
context_layer_bow = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences = True))(input_bow)

input_pose = tf.placeholder(shape=(None,MAX_SEQ_LEN,6),dtype = tf.float32)
context_layer_pose = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences = True))(input_pose)


input_gaze = tf.placeholder(shape=(None,MAX_SEQ_LEN,8),dtype = tf.float32, name='GAZE')
context_layer_gaze = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences = True))(input_gaze)


input_au = tf.placeholder(shape=(None,MAX_SEQ_LEN,35),dtype = tf.float32)
context_layer_au = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences = True))(input_au)


stacked_layer_visual = tf.stack([context_layer_bow,context_layer_pose, context_layer_gaze,context_layer_au], 
                                axis = -1)

concat_layer_visual = tf.keras.layers.Concatenate()([context_layer_bow, context_layer_pose, context_layer_gaze,
                                                     context_layer_au])

atten_dense_visual = tf.keras.layers.Dense(100)(concat_layer_visual)
atten_visual_out = tf.keras.layers.Dense(4,activation='softmax')(atten_dense_visual)

atten_visual_out_re = tf.reshape(atten_visual_out,[tf.shape(atten_visual_out)[0],tf.shape(atten_visual_out)[1],1,-1])

visual_out_fin = tf.multiply(atten_visual_out_re,stacked_layer_visual)

visual_out_fin = tf.reshape(visual_out_fin,[tf.shape(visual_out_fin)[0],tf.shape(visual_out_fin)[1],400])

fused_visual_context_layer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences = True))(visual_out_fin)

visual_fused_pool = tf.keras.layers.GlobalMaxPool1D()(fused_visual_context_layer)
fc_visual=tf.keras.layers.Dense(128,activation='relu')(visual_fused_pool)

# print("video completed")
# ################################################################## audio

# X_mfcc=tf.placeholder(tf.float32,[None,emb_dim_mfcc,78])
# # mx=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200,return_sequences=False))(X_mfcc)
# context_X_mfcc=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences=False))(X_mfcc)
# fc_X_mfcc=tf.keras.layers.Dense(100,activation='relu')(context_X_mfcc)


# X_ege=tf.placeholder(tf.float32,[None,emb_dim_ege,88])
# # mx_ege=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200,return_sequences=False))(X_ege)
# context_X_ege=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences=False))(X_ege)
# fc_X_ege=tf.keras.layers.Dense(100,activation='relu')(context_X_ege)


# X_BOAW_ege=tf.placeholder(tf.float32,[None,emb_dim_BOAW_ege,100])
# # mx_BOAW_ege=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200,return_sequences=False))(X_BOAW_ege)
# context_X_BOAW_ege=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences=False))(X_BOAW_ege)
# fc_X_BOAW_ege=tf.keras.layers.Dense(100,activation='relu')(context_X_BOAW_ege)

# X_BOAW_mfcc=tf.placeholder(tf.float32,[None,emb_dim_BOAW_mfcc,100])
# # mx_BOAW_mfcc=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200,return_sequences=False))(X_BOAW_mfcc)
# context_X_BOAW_mfcc=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences=False))(X_BOAW_mfcc)
# fc_X_BOAW_mfcc=tf.keras.layers.Dense(100,activation='relu')(context_X_BOAW_mfcc)

# # stacked_layer_audio=tf.stack([mx,mx_ege,mx_BOAW_mfcc,mx_BOAW_ege],axis=-1)#bs,100,5
# concat_layer_audio=tf.keras.layers.Concatenate()([context_X_mfcc,context_X_ege, context_X_BOAW_mfcc,context_X_BOAW_ege])
# stacked_layer_audio=tf.stack([context_X_mfcc,context_X_ege, context_X_BOAW_mfcc,context_X_BOAW_ege],axis=-1)#bs,100,5
# # # concat_layer_audio=tf.keras.layers.Concatenate()([context_X_mfcc,context_X_BOAW_mfcc])

# atten_audio_dense=tf.keras.layers.Dense(128,activation='relu')(concat_layer_audio)
# atten_audio_out=tf.keras.layers.Dense(4,activation='softmax')(atten_audio_dense)#bs,5

# out_audio_fin = tf.multiply(tf.expand_dims(atten_audio_out,1),stacked_layer_audio)
# out_audio_fin1 = tf.reshape(out_audio_fin,[tf.shape(out_audio_fin)[0],out_audio_fin.shape[1]*out_audio_fin.shape[2]])
# fc_audio=tf.keras.layers.Dense(128,activation='relu')(out_audio_fin1)

# print("audio done")
# ############################################################################# asr

X_asr=tf.placeholder(tf.float32,[None,emb_dim_asr,512])
mx_asr=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200,return_sequences=True))(X_asr)
context_X_asr=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200,return_sequences=True))(mx_asr)
context_X_asr=tf.reduce_sum(context_X_asr,1)
fc_X_asr=tf.keras.layers.Dense(128,activation='relu')(context_X_asr)
# print("transcript done")
#####################################################FUSION LAYER

fused_stacked_layer=tf.stack([fc_visual,fc_X_asr],axis=1)#bs,128,5
fused_concat_layer=tf.keras.layers.Concatenate()([fc_visual,fc_X_asr])

fused_atten_dense=tf.keras.layers.Dense(128,activation='relu')(fused_concat_layer)
fused_atten_out=tf.keras.layers.Dense(2,activation='softmax')(fused_atten_dense)

fused_out_fin = tf.multiply(tf.expand_dims(fused_atten_out,2),fused_stacked_layer)

fused_out_fin = tf.reshape(fused_out_fin,[tf.shape(fused_out_fin)[0],fused_out_fin.shape[1]*fused_out_fin.shape[2]])
fused_fc=tf.keras.layers.Dense(128,activation='relu')(fused_out_fin)
fused_fc=tf.keras.layers.Dense(64,activation='relu')(fused_fc)
fused_logits=tf.keras.layers.Dense(1)(fused_fc)
 ##############################################################################################
print("fusion done")

loss=tf.reduce_mean(tf.losses.mean_squared_error(labels=true_targets,predictions=fused_logits))
optimizer = tf.train.AdamOptimizer(learning_rate = 0.0001).minimize(loss)
print('Graph ready')

W0713 00:26:01.131720 140520442545920 deprecation.py:506] From /home/siddharth/my_env/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0713 00:26:01.136654 140520442545920 deprecation.py:506] From /home/siddharth/my_env/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0713 00:26:01.137952 140520442545920 deprecation.py:506] From /home/siddharth/my_env/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling Orthogonal.__init__ (from 

network started
fusion done
Graph ready


In [28]:
saver = tf.train.Saver()
path = "fused_model3/model.ckpt"

In [ ]:
for _ in range(5):
#     tf.reset_default_graph()
    EPOCHS = 10
    BATCH_SIZE = 10

    prev_dev_loss = 100
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    with tf.Session(config=config) as sess:

      try:
        saver.restore(sess,path)
        dev_loss = sess.run(loss,feed_dict={input_bow:dev_seq_bow,input_pose:dev_seq_pose,input_gaze:dev_seq_gaze, input_au:dev_seq_au,
                X_asr:val_data_asr,
                true_targets: dev_target})
        print('Dev Loss Currently : ',dev_loss)
        prev_dev_loss = dev_loss
      except:
        print('No pre-saved weights found')
        sess.run(tf.global_variables_initializer())

      for ep in range(EPOCHS):

        num_bts = 0
        ep_loss = 0
        tr_losses = []
        print("Epoch:",ep)
        print('Starting.. ', end = ' ')
        for j in range(len(target)//BATCH_SIZE):
    #       tr_feat = train_seq_feat[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]
          tr_pose = train_seq_pose[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]
          tr_gaze = train_seq_gaze[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]
          tr_au = train_seq_au[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]
          tr_bovw = train_seq_bow[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]

    #       tr_audio_mfcc = train_data_mfcc[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]
    #       tr_audio_ege=train_data_ege[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]
    #       tr_BOAW_mfcc=train_data_BOAW_mfcc[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]
    #       tr_BOAW_ege=train_data_BOAW_ege[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]
          tr_asr=train_data_asr[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]


          lb = target[j*BATCH_SIZE:j*BATCH_SIZE+BATCH_SIZE]
          _,l = sess.run([optimizer,loss],feed_dict={input_bow:tr_bovw,input_pose:tr_pose, input_gaze:tr_gaze, input_au:tr_au,
            X_asr:tr_asr,
            true_targets: lb}) # THIS LINE!!!
          print(l,'==',end = '')
          num_bts+=1
          ep_loss+=l
        print("=")
        ep_loss = ep_loss/num_bts
        print("Loss in epoch:{} = {}".format(ep,ep_loss))
        dev_loss = sess.run(loss,feed_dict={input_bow:dev_seq_bow,input_pose:dev_seq_pose,input_gaze:dev_seq_gaze, input_au:dev_seq_au,
                X_asr:val_data_asr,
                true_targets: dev_target})
        print("Dev Loss in epoch:{} = {}".format(ep,dev_loss))
        tr_losses.append(ep_loss)
        if dev_loss<prev_dev_loss:
          print('SAVING..')
          save_path = saver.save(sess, path)
          prev_dev_loss = dev_loss

        tr_losses.append(ep_loss)

W0713 00:26:13.196659 140520442545920 deprecation.py:323] From /home/siddharth/my_env/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Dev Loss Currently :  23.875874
Epoch: 0
Starting..  7.9192114 ==11.8700285 ==26.491695 ==45.8636 ==36.616737 ==9.855001 ==8.951131 ==3.7976074 ==7.8412085 ==12.130108 ==14.887912 ==5.491971 ==20.224636 ==10.195732 ==25.945179 ==24.663723 ===
Loss in epoch:0 = 17.046592712402344
Dev Loss in epoch:0 = 24.766494750976562
Epoch: 1
Starting..  6.8696227 ==11.472746 ==25.27811 ==42.398033 ==34.637135 ==10.375821 ==8.226828 ==3.8846676 ==7.2276163 ==12.506052 ==14.931753 ==5.6001277 ==19.00094 ==10.497817 ==26.177975 ==23.92963 ===
Loss in epoch:1 = 16.438429668545723
Dev Loss in epoch:1 = 24.94010353088379
Epoch: 2
Starting..  5.38994 ==11.51731 ==24.526684 ==35.62223 ==31.623001 ==10.50532 ==10.812901 ==4.720414 ==7.4261465 ==11.22206 ==12.834195 ==4.161871 ==14.826553 ==7.967057 ==25.0181 ==22.090633 ===
Loss in epoch:2 = 15.016526073217392
Dev Loss in epoch:2 = 25.19040870666504
Epoch: 3
Starting..  5.2640624 ==9.310728 ==23.540466 ==33.721714 ==29.84994 ==7.735513 ==9.947987 ==5.209798 